# Master Discogs Database

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from searchUtils import findExt, findSubExt, findPatternExt
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from artist import artist
from albums import albums
from album import album, albumURLInfo
from time import sleep

from masterdb import discConv, isKnownAlbum, directoryName

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-15 19:47:34.128948


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
art  = artist()
albs = albums(disc)
alb  = album()
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Create Dictionary Lookup Files

## Artist ID --> Ref and Name

In [3]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistIDMap
createArtistIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Wed Jan 15, 2020 19:47:37 for Creating Artist DBs
/Volumes/Music/Discog/artists-db/metadata/0-Metadata.p  	0 6585
/Volumes/Music/Discog/artists-db/metadata/1-Metadata.p  	1 13435
/Volumes/Music/Discog/artists-db/metadata/10-Metadata.p  	2 20112
/Volumes/Music/Discog/artists-db/metadata/11-Metadata.p  	3 26865
/Volumes/Music/Discog/artists-db/metadata/12-Metadata.p  	4 33634
/Volumes/Music/Discog/artists-db/metadata/13-Metadata.p  	5 40444
/Volumes/Music/Discog/artists-db/metadata/14-Metadata.p  	6 47019
/Volumes/Music/Discog/artists-db/metadata/15-Metadata.p  	7 53724
/Volumes/Music/Discog/artists-db/metadata/16-Metadata.p  	8 60363
/Volumes/Music/Discog/artists-db/metadata/17-Metadata.p  	9 67109
/Volumes/Music/Discog/artists-db/metadata/18-Metadata.p  	10 73843
/Volumes/Music/Discog/artists-db/metadata/19-Metadata.p  	11 80517
/Volumes/Music/Discog/artists-db/metadata/2-Metadata.p 

## Artist ID --> Albums

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Wed Jan 15, 2020 19:49:11 for Creating Artist DBs
/Volumes/Music/Discog/artists-db/metadata/0-MediaMetadata.p 	6585      3507      18202     
/Volumes/Music/Discog/artists-db/metadata/1-MediaMetadata.p 	13435     7098      37687     
/Volumes/Music/Discog/artists-db/metadata/10-MediaMetadata.p 	20112     10631     56549     
/Volumes/Music/Discog/artists-db/metadata/11-MediaMetadata.p 	26865     14189     75515     
/Volumes/Music/Discog/artists-db/metadata/12-MediaMetadata.p 	33634     17808     94436     
/Volumes/Music/Discog/artists-db/metadata/13-MediaMetadata.p 	40444     21367     113514    
/Volumes/Music/Discog/artists-db/metadata/14-MediaMetadata.p 	47019     24873     131971    
/Volumes/Music/Discog/artists-db/metadata/15-MediaMetadata.p 	53724     28412     151340    
/Volumes/Music/Discog/artists-db/metadata/16-MediaMetadata.p 	60363     31927     170301    
/Volumes/Mu

/Volumes/Music/Discog/artists-db/metadata/87-MediaMetadata.p 	582016    307520    1651635   
/Volumes/Music/Discog/artists-db/metadata/88-MediaMetadata.p 	588617    310972    1670463   
/Volumes/Music/Discog/artists-db/metadata/89-MediaMetadata.p 	595428    314568    1689658   
/Volumes/Music/Discog/artists-db/metadata/9-MediaMetadata.p 	602126    318097    1708787   
/Volumes/Music/Discog/artists-db/metadata/90-MediaMetadata.p 	608676    321524    1727612   
/Volumes/Music/Discog/artists-db/metadata/91-MediaMetadata.p 	615380    325074    1746511   
/Volumes/Music/Discog/artists-db/metadata/92-MediaMetadata.p 	622196    328673    1766341   
/Volumes/Music/Discog/artists-db/metadata/93-MediaMetadata.p 	628873    332236    1785535   
/Volumes/Music/Discog/artists-db/metadata/94-MediaMetadata.p 	635584    335720    1804197   
/Volumes/Music/Discog/artists-db/metadata/95-MediaMetadata.p 	642383    339314    1823312   
/Volumes/Music/Discog/artists-db/metadata/96-MediaMetadata.p 	649022   

## Artist ID --> Genre, Style, Artists Lookup Table

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(disc)

## Album ID --> Name, Ref, Artists Lookup Table

In [6]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createAlbumIDMap
createAlbumIDMap(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Mon Jan 13, 2020 23:52:12 for Creating Artist DBs
/Volumes/Music/Discog/albums-db/metadata/0-ArtistAlbums.p 	21795
/Volumes/Music/Discog/albums-db/metadata/1-ArtistAlbums.p 	41389
/Volumes/Music/Discog/albums-db/metadata/10-ArtistAlbums.p 	67701
/Volumes/Music/Discog/albums-db/metadata/11-ArtistAlbums.p 	89643
/Volumes/Music/Discog/albums-db/metadata/12-ArtistAlbums.p 	116448
/Volumes/Music/Discog/albums-db/metadata/13-ArtistAlbums.p 	128920
/Volumes/Music/Discog/albums-db/metadata/14-ArtistAlbums.p 	152257
/Volumes/Music/Discog/albums-db/metadata/15-ArtistAlbums.p 	174121
/Volumes/Music/Discog/albums-db/metadata/16-ArtistAlbums.p 	192193
/Volumes/Music/Discog/albums-db/metadata/17-ArtistAlbums.p 	205799
/Volumes/Music/Discog/albums-db/metadata/18-ArtistAlbums.p 	230255
/Volumes/Music/Discog/albums-db/metadata/19-ArtistAlbums.p 	250482
/Volumes/Music/Discog/albums-db/metadata/2-Artis

# Master Lookup Tests

In [ ]:
%load_ext autoreload
%autoreload
from lookup import testLookupMaps
testLookupMaps(disc)

# Pandas DB

## Slim Artist DB

In [ ]:
from masterdb import getSlimArtistDB

In [ ]:
discSlimArtistDB = getSlimArtistDB(disc)
discSlimArtistDB.head()

## Artist DB

In [11]:
from masterdb import getArtistDB

In [12]:
discArtistDB = getArtistDB(disc)
discArtistDB.head()

Current Time is Thu Jan 02, 2020 12:21:09 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 12.1MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 14.4MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToVariations.p
  --> This file is 24.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToVariations.p
Creating Pandas DataFrame for 636524 Artists
	Shape --> (636524, 1)
  Joining Ref
	Shape --> (636524, 2)
  Joining Variations
	Shape --> (636524, 3)
  Finding Real Artist Name
	Shape --> (636524, 6)
DataFrame Shape is (636524, 6)
Current Time is Thu Jan 02, 2020 12:21:46 for Done with 
=================================== Creating Artist DB ===================================
Process [{0}] took 37 seconds.
Saving Mas

,Name,Ref,Variations,Known,Artist,Num
1000,Dave Clarke,/artist/1000-Dave-Clarke,"[Dave Clarke, 7 Red 7, Clarck, Clarke, Clarke,...",True,Dave Clarke,NaN
1000500,Club Pulse,/artist/1000500-Club-Pulse,[Club Pulse],True,Club Pulse,NaN
100200,Dike,/artist/100200-Dike,"[Dike, D.I.K.E., Dike D, Uchegdu]",True,Dike,NaN
1002000,Larry Stokes,/artist/1002000-Larry-Stokes,[Larry Stokes],True,Larry Stokes,NaN
1002600,Gidd Sanchez,/artist/1002600-Gidd-Sanchez,[Gidd Sanchez],True,Gidd Sanchez,NaN


## Artist Metadata DB

In [13]:
from masterdb import getArtistMetadataDB

In [14]:
discArtistMetadataDB = getArtistMetadataDB(disc)
discArtistMetadataDB.head()

Current Time is Thu Jan 02, 2020 12:21:59 for 
=================================== Creating Artist Metadata DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToGenre.p
  --> This file is 10.3MB.
Loading /Volumes/Music/Discog/db/ArtistIDToGenre.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToStyle.p
  --> This file is 15.1MB.
Loading /Volumes/Music/Discog/db/ArtistIDToStyle.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToCollaborations.p
  --> This file is 33.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToCollaborations.p
Creating Pandas DataFrame for 489596 Artists
	Shape --> (489596, 1)
  Joining Style
	Shape --> (489596, 2)
  Joining Collaboration
	Shape --> (489596, 3)
DataFrame Shape is (489596, 3)
Current Time is Thu Jan 02, 2020 12:23:12 for Done with 
=================================== Creating Artist Metadata DB ===================================
Process [{0}] took 1.2 minutes.
Saving Master Artist M

,Genre,Style,Collaboration
1000,"{'Electronic': 54, 'Hip Hop': 1, 'Non-Music': ...","{'Techno': 45, 'Electro': 16, 'Downtempo': 1, ...","{'Dave Clarke': 53, 'Bang The Future': 1, 'Mr...."
1000500,{'Electronic': 1},{'Progressive Trance': 1},{'Club Pulse': 1}
100200,{'Hip Hop': 11},"{'Conscious': 7, 'Pop Rap': 2, '': 3}","{'Dike': 11, 'ABS (2)': 2, 'Creutzfeld&Jakob':..."
1002000,{},{},{}
1005400,{'Rock': 5},"{'Alternative Rock': 1, 'Indie Rock': 1, 'Math...","{'Gapeseed': 5, 'Gerling': 1}"


## Artist Albums DB

In [15]:
from masterdb import getArtistAlbumsDB

In [16]:
discArtistAlbumsDB = getArtistAlbumsDB(disc)
discArtistAlbumsDB.head()

Current Time is Thu Jan 02, 2020 12:23:42 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
  --> This file is 174.0MB.
Loading /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
Creating Pandas DataFrame for 636524 Artists
	Shape --> (636524, 1)
DataFrame Shape is (636524, 1)
Current Time is Thu Jan 02, 2020 12:25:23 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [{0}] took 1.7 minutes.


,Albums
1000,"{'Albums': {'44815': 'Archive One', '44831': '..."
1000500,{'Miscellaneous': {'1178875': 'Peak Controller'}}
100200,"{'Albums': {'50511': 'PottpÃ¼ree'}, 'Singles &..."
1002000,{'Instruments & Performance': {'1002108': 'Are...
1002600,{'Vocals': {'1180905': 'Quiero'}}


## Albums DB

In [17]:
from masterdb import getAlbumDB

In [18]:
discAlbumDB = getAlbumDB(disc)
discAlbumDB.head()

Current Time is Thu Jan 02, 2020 12:25:23 for 
=================================== Creating Artist Album DB ===================================
Loading AlbumID Data
Loading data from /Volumes/Music/Discog/db/AlbumIDToName.p
  --> This file is 35.5MB.
Loading /Volumes/Music/Discog/db/AlbumIDToName.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToRef.p
  --> This file is 49.9MB.
Loading /Volumes/Music/Discog/db/AlbumIDToRef.p
Loading data from /Volumes/Music/Discog/db/AlbumIDToArtists.p
  --> This file is 20.2MB.
Loading /Volumes/Music/Discog/db/AlbumIDToArtists.p
Creating Pandas DataFrame for 1521843 Albums
	Shape --> (1521843, 1)
  Joining Ref
	Shape --> (1521843, 2)
  Joining Artists
	Shape --> (1521843, 3)
DataFrame Shape is (1521843, 3)
Current Time is Thu Jan 02, 2020 12:26:37 for Done with 
=================================== Creating Artist Album DB ===================================
Process [{0}] took 1.2 minutes.


,Name,Ref,Artists
1012077,Live,/Dave-Clarke-Live/master/1012077,[1000]
1258261,Walls Of Genius And Miracle,/Walls-Of-Genius-and-Miracle-Walls-Of-Genius-A...,"[1000, 1009227, 182635]"
2546978,RA.EX007 Dave Clarke,/Dave-Clarke-RAEX007-Dave-Clarke/release/2546978,[1000]
44815,Archive One,/Dave-Clarke-Archive-One/master/44815,[1000]
44831,Devil's Advocate,/Dave-Clarke-Devils-Advocate/master/44831,[1000]


## Artist Album ID --> Known Albums

In [19]:
from masterdb import getArtistAlbumKnownDB

In [20]:
discArtistAlbumKnownDB = getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB)
discArtistAlbumKnownDB.head()

Current Time is Thu Jan 02, 2020 12:26:37 for 
=================================== Creating Artist Album DB ===================================
Creating Pandas DataFrame for 636524 Arist Albums
	Shape --> (636524, 3)
DataFrame Shape is (636524, 3)
Current Time is Thu Jan 02, 2020 12:27:23 for Done with 
=================================== Creating Artist Album DB ===================================
Process [{0}] took 46 seconds.


,Known Albums,All Albums,Albums
1000,53,53,"{'44815': ['Archive One', 'Albums', True], '44..."
1000500,1,1,"{'1178875': ['Peak Controller', 'Miscellaneous..."
100200,9,9,"{'50511': ['PottpÃ¼ree', 'Albums', True], '219..."
1002000,3,3,"{'1002108': ['Are You Faithful?', 'Instruments..."
1002600,1,1,"{'1180905': ['Quiero', 'Vocals', True]}"


# Joining Artist ID DataFrame

In [ ]:
from masterdb import createMasterDB

In [ ]:
createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB)

In [ ]:
from random import random